In [1]:
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, \
    ReduceLROnPlateau, TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop

from utils import get_imagens_erro, report, plot_errors
import time

MODEL_DIR = '../modelschest'
LOG_DIR = '../logs/02d/chestXRay'
SIZE = (224, 224)
caminho_bases = os.path.join('..', 'bases', 'chest_xray')
caminho_train = os.path.join(caminho_bases, 'train')
caminho_test = os.path.join(caminho_bases, 'test')

# Callbacks

In [2]:
tensorboard_logs = TensorBoard(log_dir=LOG_DIR, histogram_freq=1,
                               write_graph=False, write_images=False,
                               update_freq='epoch')
mcp_save = ModelCheckpoint(os.path.join(MODEL_DIR, 
                                        'Transfermodelweights02d.{epoch:02d}-{val_loss:.2f}.hdf5'),
                           save_best_only=True, monitor='val_loss', mode='min')
early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=4,
                              verbose=1, min_delta=1e-2, mode='min')


# Load features to np

In [3]:
FEATURES_DIR = os.path.join('..', 'bases', 'chestXRay_02cfeatures')
with open(os.path.join(FEATURES_DIR, 'features_train.np'), 'rb') as features_in:
    features_train = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_train.np'), 'rb') as y_in:
    y_train = np.load(y_in)
with open(os.path.join(FEATURES_DIR, 'features_test.np'), 'rb') as features_in:
    features_test = np.load(features_in)
with open(os.path.join(FEATURES_DIR, 'y_test.np'), 'rb') as y_in:
    y_test = np.load(y_in)

# Model

In [4]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256,
                        activation='relu',
                        kernel_regularizer=tf.keras.regularizers.l1_l2(l2=0.01, l1=0.01),
                        kernel_initializer='glorot_normal',
                        input_shape=(1024,)),
  tf.keras.layers.Dropout(0.33),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [5]:
model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               262400    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 257       
Total params: 262,657
Trainable params: 262,657
Non-trainable params: 0
_________________________________________________________________


# Training and validation

In [30]:
epochs = 20

def get_epoch(history, number):
    acc = history['acc'][number]
    val_acc = history['val_acc'][number]
    return 'Number: %s - acc: %s  - val_acc: %s' % (number, acc, val_acc)
    

def resume_epochs(history, batch_size):
    print('Optimizer: %s lr: %s Batch Size: %s' % (optimizer_function.__name__, lr, batch_size))
    print( [get_epoch(history.history, 0), 
            get_epoch(history.history, len(history.history) // 2),
            get_epoch(history.history, len(history.history) - 1)
           ])

best_val_acc = 0
best_params = None
all_historys = []
for lr in [0.01, 0.001, 0.0004, 0.0001, 0.00004, 0.00002, 0.00001]:
    for optimizer_function in [RMSprop, Adam]:
        for batch_size in [64, 128, 256]:
            optimizer=optimizer_function(lr=lr)
            model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
            history = model.fit(x=features_train,
                                y=y_train,
                                batch_size=batch_size,
                                epochs=epochs,
                                verbose=0,
                                validation_data=(features_test, y_test),
                               )
            resume_epochs(history, batch_size)
            this_history = {'lr': lr,
                           'epochs': ind,
                           'optimizer': optimizer_function.__name__,
                           'val_acc': val_acc,
                           'batch_size': batch_size,
                           'history': history}
            all_historys.append(this_history)
            for ind, val_acc in enumerate(history.history['val_acc'], 1):
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_params = this_history


Optimizer: RMSprop lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.66512346  - val_acc: 0.61513156', 'Number: 2 - acc: 0.7212577  - val_acc: 0.84046054', 'Number: 3 - acc: 0.75906634  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.5364583  - val_acc: 0.3848684', 'Number: 2 - acc: 0.59799385  - val_acc: 0.3848684', 'Number: 3 - acc: 0.63638115  - val_acc: 0.61513156']
Optimizer: RMSprop lr: 0.01 Batch Size: 256
['Number: 0 - acc: 0.6435185  - val_acc: 0.61513156', 'Number: 2 - acc: 0.7195216  - val_acc: 0.61513156', 'Number: 3 - acc: 0.69290125  - val_acc: 0.61513156']
Optimizer: Adam lr: 0.01 Batch Size: 64
['Number: 0 - acc: 0.837963  - val_acc: 0.85855263', 'Number: 2 - acc: 0.8686343  - val_acc: 0.76644737', 'Number: 3 - acc: 0.86902004  - val_acc: 0.8256579']
Optimizer: Adam lr: 0.01 Batch Size: 128
['Number: 0 - acc: 0.8157793  - val_acc: 0.75986844', 'Number: 2 - acc: 0.8771219  - val_acc: 0.8256579', 'Number: 3 - acc: 0.8715278  - val_acc: 

Optimizer: Adam lr: 1e-05 Batch Size: 256
['Number: 0 - acc: 0.93441355  - val_acc: 0.8684211', 'Number: 2 - acc: 0.9361497  - val_acc: 0.86019737', 'Number: 3 - acc: 0.9376929  - val_acc: 0.8569079']


In [31]:
best_params

{'batch_size': 256,
 'epochs': 20,
 'history': <tensorflow.python.keras.callbacks.History at 0x7f3dcb69a4e0>,
 'lr': 0.0001,
 'optimizer': 'RMSprop',
 'val_acc': 0.8355263}

In [29]:
best_params

{'batch_size': 128,
 'epochs': 18,
 'lr': 0.0004,
 'optimizer': 'Adam',
 'val_acc': 0.8930921}

In [32]:
all_historys

[{'batch_size': 64,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd9814630>,
  'lr': 0.01,
  'optimizer': 'RMSprop',
  'val_acc': 0.8552632},
 {'batch_size': 128,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd1524f60>,
  'lr': 0.01,
  'optimizer': 'RMSprop',
  'val_acc': 0.84375},
 {'batch_size': 256,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd47df278>,
  'lr': 0.01,
  'optimizer': 'RMSprop',
  'val_acc': 0.61513156},
 {'batch_size': 64,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd1dd6f60>,
  'lr': 0.01,
  'optimizer': 'Adam',
  'val_acc': 0.61513156},
 {'batch_size': 128,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd57ae0b8>,
  'lr': 0.01,
  'optimizer': 'Adam',
  'val_acc': 0.7746711},
 {'batch_size': 256,
  'epochs': 20,
  'history': <tensorflow.python.keras.callbacks.History at 0x7f3dd0947748>,
  'lr

In [33]:
import pickle

In [34]:
pickle.dumps(all_historys)

TypeError: can't pickle SwigPyObject objects

In [ ]:
all_historys_dict = {}